#####  环境：阿里云 ubuntu  python27 tensorflowf1.4.0

In [2]:
import tensorflow as tf

In [3]:
tf.__version__

'1.4.0'

### 3.1.2 计算图的使用

#### Tensorflow程序执行分两步：定义计算图所有的计算，以及执行计算，如下cell是第一步

In [6]:
a = tf.constant([1.0,2.0],name="a")
b = tf.constant([2.0,3.0],name="b")
result = a+b

#### 获取默认计算图，并查看一个运算所属的计算图

In [9]:
print(a.graph is tf.get_default_graph())

True


#### 还支持tf.Graph函数生成新的计算图，不同计算图上的张量和运算不共享

In [12]:
import tensorflow as tf

g1 = tf.Graph()
with g1.as_default():
    v = tf.get_variable("v",shape=[1],initializer=tf.zeros_initializer)
    
g2 = tf.Graph()
with g2.as_default():
    v = tf.get_variable("v",shape=[1],initializer=tf.ones_initializer)

with tf.Session(graph=g1) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("",reuse=True):
        print(sess.run(tf.get_variable("v")))

with tf.Session(graph=g2) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("",reuse=True):
        print(sess.run(tf.get_variable("v")))

[0.]
[1.]


## 3.2 Tensorflow 数据模型-张量

## 3.3 TensorFlow运行模型-会话

In [18]:
a = tf.constant([1.0,2.0],name="a")
b = tf.constant([2.0,3.0],name="b")
result = a+b

sess = tf.Session()
result_out = sess.run(result)
sess.close()

print(result_out)

[3. 5.]


#### 如上cell中，需要明确调用Session.close()函数关闭并释放资源。但是程序因异常退出的时候，关闭函数就可能不会被执行而导致资源泄露。
#### 为解决该问题，采用python上下文管理器来管理会话。

In [20]:
a = tf.constant([1.0,22.0],name="a")
b = tf.constant([4.0,3.0],name="b")
result = a+b

with tf.Session() as sess:
    sess.run(result)

####  计算张量的取值

In [21]:
sess = tf.Session()
with sess.as_default():
    print(result.eval())

[ 5. 25.]


In [22]:
#  上面cell代码的另一种实现
sess = tf.Session()

print(sess.run(result))
print(result.eval(session=sess))

[ 5. 25.]
[ 5. 25.]


#### 在交互式环境下可以直接构建默认会话

In [23]:
sess = tf.InteractiveSession()
print(result.eval())
sess.close()

[ 5. 25.]


## 3.4 TensorFlow实现神经网络

In [24]:
# 一个变量在被使用之前，这个变量的初始化过程需要被明确地调用
weights = tf.Variable(tf.random_normal([2,3],stddev=2))

w2 = tf.Variable(weights.initialized_value())
w3 = tf.Variable(weights.initialized_value()*2.0)

In [47]:
import tensorflow as tf
w1 = tf.Variable(tf.random_normal((2,3),stddev=1,seed=1))
w2 = tf.Variable(tf.random_normal((3,1),stddev=1,seed=1))

x = tf.constant([[0.7,0.9]])

a = tf.matmul(x,w1)
y = tf.matmul(a,w2)

sess = tf.Session()

# error：sess.run(w1.initialized_value()) 
sess.run(w1.initializer)
sess.run(w2.initializer)
'''
# 如上对w1和w2单独进行初始化，现在可通过下面语句直接对着两个变量初始化
init_op = tf.global_variables_initializer()
sess.run(init_op)
'''
print(w1.eval(session=sess))
print('y=%f'%(sess.run(y)))
sess.close()

[[-0.8113182   1.4845988   0.06532937]
 [-2.4427042   0.0992484   0.5912243 ]]
y=3.957578


#### 一个变量在构建之后，它的类型就不能再改变了

In [50]:
w1 = tf.Variable(tf.random_normal([2,3],stddev=1,seed=1),name="w1")
w2 = tf.Variable(tf.random_normal((2,3),dtype = tf.float64,stddev=1,seed=1),name="w2")

In [52]:
print(w1,w2)

(<tf.Variable 'w1_2:0' shape=(2, 3) dtype=float32_ref>, <tf.Variable 'w2_2:0' shape=(2, 3) dtype=float64_ref>)


In [ ]:
w1.assign(w2) # 会报错

#### 维度在程序运行中是有可能改变的，但是要设置validate_shape = False

In [56]:
w1 = tf.Variable(tf.random_normal([2,3],stddev=1,seed=1),name="w1")
w2 = tf.Variable(tf.random_normal([2,2],stddev=1,seed=1),name="w2")

In [58]:
# tf.assign(w1,w2) #会报错

In [59]:
 tf.assign(w1,w2,validate_shape = False) 

<tf.Tensor 'Assign_2:0' shape=(2, 2) dtype=float32_ref>

### 3.4.4 通过TensorFlow训练神经网络模型

In [60]:
import tensorflow as tf
w1 = tf.Variable(tf.random_normal((2,3),stddev=1,seed=1))
w2 = tf.Variable(tf.random_normal((3,1),stddev=1,seed=1))

x = tf.placeholder(tf.float32,shape=(1,2),name="input")
a = tf.matmul(x,w1)
y = tf.matmul(a,w2)

sess = tf.Session()

init_op = tf.global_variables_initializer()
sess.run(init_op)

print(sess.run(y,feed_dict={x:[[0.7,0.9]]}))
sess.close()

[[3.957578]]


In [ ]:
y = tf.sigmoid(y)
cross_entropy = -tf.reduce_mean(
    y_*tf.log(tf.clip_by_value(y,1e-10,1.0))
    +(1-y_)*tf.log(tf.clip_by_value(1-y,1e-10,1.0))
)

learning_rate = 0.001
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

### 3.4.5 完整神经网络样例程序

In [65]:
import tensorflow as tf

from numpy.random import RandomState

batch_size = 8

w1 = tf.Variable(tf.random_normal((2,3),stddev=1,seed=1))
w2 = tf.Variable(tf.random_normal((3,1),stddev=1,seed=1))

#在shape维度使用none方便使用不同batch的大小
x = tf.placeholder(tf.float32,shape=(None,2),name="x-input")
y_ = tf.placeholder(tf.float32,shape=(None,1),name="y-input")

# 前向
a = tf.matmul(x,w1)
y = tf.matmul(a,w2)

# 定义损失函数和反向传播
y = tf.sigmoid(y)
cross_entropy = -tf.reduce_mean(
    y_*tf.log(tf.clip_by_value(y,1e-10,1.0))
    +(1-y_)*tf.log(tf.clip_by_value(1-y,1e-10,1.0))
)

learning_rate = 0.001
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

#通过随机数生成数据集
rdm = RandomState(1)
dataset_size = 128
X = rdm.rand(dataset_size,2)
#定义规则样本标签。所有x1+x2<1认为是正样本表示为1，负样本表示为0
Y = [[int(x1+x2<1) ]for (x1,x2) in X]

with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    #初始化变量
    sess.run(init_op)
    
    print sess.run(w1)
    print sess.run(w2)
    
    STEPS = 5000
    for i in range(STEPS):
        #每次选取batch个样本进行训练
        start = (i*batch_size)%dataset_size
        end = min(start+batch_size,dataset_size)
        
        #通过选取的样本外训练神经网络并更新参数
        sess.run(train_step,feed_dict={x:X[start:end],y_:Y[start:end]})
        if i%1000 == 0:
            total_cross_entropy = sess.run(cross_entropy,feed_dict={x:X,y_:Y})
            print("After %d training step(s),cross entropy on all data is %g"%\
                 (i,total_cross_entropy))


[[-0.8113182   1.4845988   0.06532937]
 [-2.4427042   0.0992484   0.5912243 ]]
[[-0.8113182 ]
 [ 1.4845988 ]
 [ 0.06532937]]
After 0 training step(s),cross entropy on all data is 1.89805
After 1000 training step(s),cross entropy on all data is 0.655075
After 2000 training step(s),cross entropy on all data is 0.626172
After 3000 training step(s),cross entropy on all data is 0.615096
After 4000 training step(s),cross entropy on all data is 0.610309


RuntimeError: Attempted to use a closed Session.